## 필요한 라이브러리 불러오기 및 상수 선언

In [1]:
import pandas as pd
import glob
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 앞뒤로 각각 몇 개의 데이터와 비교할지
NUM_NEIGHBORS = 10

In [32]:
LDA = pd.read_csv("LabeledData_added.csv")

In [33]:
LDA.drop(['ThumbnailUrl_x', 'label', 'ThumbnailUrl_y'], axis=1, inplace=True)
LDA

,Unnamed: 0,VideoId,Published,ViewCount,HDThumbnailUrl,Title
0,0,-4BvjRGDIAw,2019-01-03T10:00:00Z,38478.0,https://i.ytimg.com/vi/-4BvjRGDIAw/sddefault.jpg,"【히오스/하이라이트】 새해맞이 행복시공 (feat. Rich, KyoCha)"
1,1,HTpd6iJdxyk,2019-01-03T10:00:01Z,69165.0,https://i.ytimg.com/vi/HTpd6iJdxyk/sddefault.jpg,【히오스】 2019년에도 무서운 침아블로 (feat. Rich)
2,2,Tf7lwAJirNY,2019-01-04T08:40:26Z,98047.0,https://i.ytimg.com/vi/Tf7lwAJirNY/sddefault.jpg,【다크 세라핌】 1화 - 추락천사 알카드
3,3,l1LT2hccFfI,2019-01-04T10:00:08Z,85222.0,https://i.ytimg.com/vi/l1LT2hccFfI/sddefault.jpg,【히오스】 단 8분 46초만에 끝내는 필승전략
4,4,3cpJ6wSC3xI,2019-01-05T13:04:41Z,398331.0,https://i.ytimg.com/vi/3cpJ6wSC3xI/sddefault.jpg,【메탈슬러그 - 침착맨X주호민】 인해전술 공략법
...,...,...,...,...,...,...
1856,1856,8hSds3puPQY,2023-02-10T10:00:09Z,304385.0,https://i.ytimg.com/vi/8hSds3puPQY/sddefault.jpg,(잔인주의) 선량한 사람은 너굴맨이 처리했으니 안심하라구
1857,1857,PhWyC9sPF0M,2023-02-11T10:00:11Z,346931.0,https://i.ytimg.com/vi/PhWyC9sPF0M/sddefault.jpg,모든 분리수거의 신
1858,1858,a6Kw3GQxcKg,NaN,NaN,https://i.ytimg.com/vi/a6Kw3GQxcKg/sddefault.jpg,밀덕이 훌륭한 지휘관이 될 수 있을까?
1859,1859,h2LwNXWnsz4,NaN,NaN,https://i.ytimg.com/vi/h2LwNXWnsz4/sddefault.jpg,야끼도리동과 고프로로 찍은 영상들


In [34]:
LDA.dropna(inplace=True)
LDA

,Unnamed: 0,VideoId,Published,ViewCount,HDThumbnailUrl,Title
0,0,-4BvjRGDIAw,2019-01-03T10:00:00Z,38478.0,https://i.ytimg.com/vi/-4BvjRGDIAw/sddefault.jpg,"【히오스/하이라이트】 새해맞이 행복시공 (feat. Rich, KyoCha)"
1,1,HTpd6iJdxyk,2019-01-03T10:00:01Z,69165.0,https://i.ytimg.com/vi/HTpd6iJdxyk/sddefault.jpg,【히오스】 2019년에도 무서운 침아블로 (feat. Rich)
2,2,Tf7lwAJirNY,2019-01-04T08:40:26Z,98047.0,https://i.ytimg.com/vi/Tf7lwAJirNY/sddefault.jpg,【다크 세라핌】 1화 - 추락천사 알카드
3,3,l1LT2hccFfI,2019-01-04T10:00:08Z,85222.0,https://i.ytimg.com/vi/l1LT2hccFfI/sddefault.jpg,【히오스】 단 8분 46초만에 끝내는 필승전략
4,4,3cpJ6wSC3xI,2019-01-05T13:04:41Z,398331.0,https://i.ytimg.com/vi/3cpJ6wSC3xI/sddefault.jpg,【메탈슬러그 - 침착맨X주호민】 인해전술 공략법
...,...,...,...,...,...,...
1853,1853,nRBLhMZjazk,2023-02-07T10:00:27Z,362440.0,https://i.ytimg.com/vi/nRBLhMZjazk/sddefault.jpg,헛리버 공국(Principality of Hutt River)을 알아보자
1854,1854,JgkdgAfTdBk,2023-02-08T10:00:21Z,690444.0,https://i.ytimg.com/vi/JgkdgAfTdBk/sddefault.jpg,성우 남도형 초대석
1855,1855,mdsaZz5WVHU,2023-02-09T10:00:34Z,627915.0,https://i.ytimg.com/vi/mdsaZz5WVHU/sddefault.jpg,계속되는 남도형 성우와의 토크
1856,1856,8hSds3puPQY,2023-02-10T10:00:09Z,304385.0,https://i.ytimg.com/vi/8hSds3puPQY/sddefault.jpg,(잔인주의) 선량한 사람은 너굴맨이 처리했으니 안심하라구


In [35]:
LDA.to_csv("LabeledData_침착맨.csv")

## 데이터셋 가져오기

In [19]:
# PATH 설정
path = r'C:\Users\sungh\OneDrive\Desktop\THUMBNAILS\test'

In [20]:
# dataset 하나로 합치기
all_files = glob.glob(os.path.join(path, "*.csv"))

In [21]:
df_all = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [23]:
# index 초기화 및 불필요한 column drop
df_all.reset_index(inplace=True)
df_all.drop(['index', 'Unnamed: 0'], axis=1, inplace=True)

In [24]:
df_all.isnull().sum()

level_0           0
Published         0
HDThumbnailUrl    0
ViewCount         0
VideoId           0
Title             0
dtype: int64

In [18]:
df_all[df_all['Published'].isna()]

,level_0,Published,HDThumbnailUrl,ViewCount,VideoId,Title
3373,3373,NaN,https://i.ytimg.com/vi/a6Kw3GQxcKg/sddefault.jpg,NaN,a6Kw3GQxcKg,밀덕이 훌륭한 지휘관이 될 수 있을까?
3374,3374,NaN,https://i.ytimg.com/vi/h2LwNXWnsz4/sddefault.jpg,NaN,h2LwNXWnsz4,야끼도리동과 고프로로 찍은 영상들
3375,3375,NaN,https://i.ytimg.com/vi/vxtFplAvnsM/sddefault.jpg,NaN,vxtFplAvnsM,김치순두제비는 언제 먹나요?


In [ ]:
# @seah가 만든 labeling함수.
# 앞뒤 비교할 n개의 평균과 조회수를 비교하여 더 높으면 1, 낮으면 0을 칼럼에 저장한다.

def getLabels(df, n) :

    # input : dataset to get labels, number of neighbors to get mean of views
    # output : dataset with labels and mean of neighbors

    # create columns 'NeighborsMean' & 'label'
    df['NeighborsMean'] = 0
    df['label'] = 0
    df = df.astype({'NeighborsMean':'int'})

    for i in range(len(df)):
        mean = 0

        # [ : NUM_NEIGHBORS - 1 ]
        if i in range(0, n):
            mean = df.iloc[:i+n+1]['ViewCount'].mean()
        
        # [ NUM_NEIGHBORS : length - NUM_NEIGHBORS - 1 ]
        elif i in range(n, len(df)-n):
            mean = df.iloc[i-n:i+n+1]['ViewCount'].mean()
        
        # [ length - NUM_NEIGHBORS : ]
        else:
            mean = df.iloc[i-n:]['ViewCount'].mean()
        
        df['NeighborsMean'][i] = round(mean)
        if df['ViewCount'][i] >= mean:
            df['label'][i] = 1
    
    return df

In [ ]:
df_all = getLabels(df_all, NUM_NEIGHBORS)
df_all.drop(['Published', 'ViewCount', 'NeighborsMean'], axis=1, inplace=True)
df_all

In [ ]:
df_all['label'].value_counts()

In [ ]:
df_all.to_csv("LabeledData.csv")